In [0]:
import dlt
from pyspark.sql.functions import col, regexp_extract, input_file_name
import os

# Récupération de la clé stockée dans le scope
key = dbutils.secrets.get(scope="ln-scope", key="AZURE_STORAGE_KEY")

spark.conf.set(
  "fs.azure.account.key.lndqualiteeaustorage.dfs.core.windows.net",
  key
)
# --- Table bronze COM ---
# Récupère la liste des fichiers commençant par "DIS_COM_UDI"
files_com = [
    f.path
    for f in dbutils.fs.ls("abfss://lnd-fichiers-dezip@lndqualiteeaustorage.dfs.core.windows.net/")
    if f.name.startswith("DIS_COM_UDI")
]
print(files_com)

@dlt.table(
    comment="Table bronze avec les données brutes depuis ADLS les fichiers DIS_COM_UDI_annee.txt",
    partition_cols=["annee"]
)
def bronze_com():
    df = (
        spark.read.format("csv")
        .option("header", "true")
        .load(files_com)
    )
    # Extraire l'année depuis le nom du fichier (ex: DIS_COM_UDI_2023.txt)
    df = df.withColumn(
        "annee",
        regexp_extract(col("_metadata.file_path"), r"DIS_COM_UDI_(\d{4})", 1)
    )
    return df

# --- Table bronze PLV ---
# Récupère la liste des fichiers commençant par "DIS_PLV"
files_plv = [
    f.path
    for f in dbutils.fs.ls("abfss://lnd-fichiers-dezip@lndqualiteeaustorage.dfs.core.windows.net/")
    if f.name.startswith("DIS_PLV")
]
print(files_plv)

@dlt.table(
    comment="Table bronze avec les données brutes depuis ADLS les fichiers DIS_PLV_annee_xxx.txt",
    partition_cols=["annee"]
)
def bronze_plv():
    df = (
        spark.read.format("csv")
        .option("header", "true")
        .load(files_plv)
    )
    # Extraire l'année depuis le nom du fichier (ex: DIS_PLV_2024_xxx.txt)
    df = df.withColumn(
        "annee",
        regexp_extract(col("_metadata.file_path"), r"DIS_PLV_(\d{4})", 1)
    )
    return df

